<a href="https://colab.research.google.com/github/ykim879/soccer.webscrapping/blob/master/Soccer_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import threading
import queue
baseURL = 'https://www.soccerbase.com'
lock = threading.Lock()

In [ ]:
#Decorator hopefully
import time
def timer(func):
  def timeIt(*args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    lock.acquire()
    print(func.__name__, 'took', end = ' ')
    print(f'{time.time() - start:0.2f}', 'seconds to run.')
    lock.release()
  return timeIt

In [ ]:
@timer
def getEveryCompetitionData():
  request = requests.get("https://www.soccerbase.com/players/home.sd")
  soup = BeautifulSoup(request.content, "lxml")
  testing = soup.find('div', id = 'players-menu')
  for div in testing.find_all('div', class_ = 'soccerColumn'):
    if (re.search('Competition', div.text) != None):
      comp = div
  threads = []
  compQ = queue.Queue()
  for competition in comp.find_all('a'):
    t = threading.Thread(target = readCompetitionPage, args = (competition.text, baseURL + competition['href'], compQ))
    threads.append(t)
    t.start()
  for t in threads:
    t.join()
  counter = 1
  while not compQ.empty():
    #print(counter, compQ.get()[0])
    counter += 1

#From competition page of teams
## seasons: dataframe consisted of each season value with the season date

#### getTeamData(team: teamname, url: url has to go to, queue: consisted of map)
goal: 

In [ ]:
class Team:
  def __init__(self, team_name, players_df):
    self.team_name = team_name
    self.players_df = players_df
  def get_name(self):
    return self.team_name
  def get_players(self):
    return self.players_df

In [ ]:
import pandas as pd
#From player page for a team
def getTeamData(team, url, q):
  #names = [team]
  players_df = pd.DataFrame()
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  try:
    test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
    for elem in test.find_all('b'):
      name = elem.find('a')
      players_df = players_df.append({"link": name["href"], "name" : name.text}, ignore_index = True)
  except AttributeError:
      print(team, 'failed to open')
  team = Team(team, players_df)
  q.put(team)

def getTeamData(team, url):
  #names = [team]
  players_df = pd.DataFrame()
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  try:
    test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
    for elem in test.find_all('b'):
      name = elem.find('a')
      players_df = players_df.append({"link": name["href"], "name" : name.text}, ignore_index = True)
  except AttributeError:
      print(team, 'failed to open')
  return Team(team, players_df)

#### season_selector
teams: map with key is a team name and value is dataframe of player information

In [ ]:
def season_selector(seasons):

  value = seasons.iloc[1]["value"]

  url = "https://www.soccerbase.com/tournaments/tournament.sd?tourn_id=" + value
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  result = []
  try:
    teamNames = soup.find("table", class_ = "table")
    team_df = teams_dataframes(teamNames)
    #result.extend(outputTeams(team_df))
    result.append(getTeamData(team_df.iloc[0].index, baseURL + team_df.iloc[0]["link"]))
    return result
  except AttributeError:
    pass

In [ ]:
def outputTeams(team_df):
  teams = []
  threads = []
  q = queue.Queue()
  for index, row in team_df.iterrows():
    t = threading.Thread(target = getTeamData, args = (index, "https://www.soccerbase.com" + row["link"], q))
    threads.append(t)
    t.start()
  for t in threads:
    t.join()
  while not q.empty():
    teams.append(q.get())
  return teams

In [ ]:
import threading
import queue
#@timer
def readCompetitionPage(competition, compURL, compQ):
  request = requests.get(compURL)
  soup = BeautifulSoup(request.content, "lxml")
  seasons = season_dataframes(soup)
  teams = season_selector(seasons)
  if teams != None:
    for team in teams:
      print(team.get_players())

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def getPlayerData(url, name): # create new dataframe because we don't want dependency
  request = requests.get(baseURL + url)
  soup = BeautifulSoup(request.content, "lxml")
  body = soup.find("tr", class_ = "alt")
  information = []
  for info in body.find_all("td"):
    information.append(info.text)
  row = pd.DataFrame()
  row = row.append({"name": name, "url": url, "recent team": information[0], "from": information[1],
                    "to": information[2], "fee": information[3]}, ignore_index = True)
  return row
# test for getPlayerData
pd.set_option("display.max_columns",6)
print(getPlayerData("/players/player.sd?player_id=61390", "Bernd Leno"))

             fee          from        name recent team to  \
0   £19,300,000    19 Jun, 18   Bernd Leno     Arsenal      

                                  url  
0  /players/player.sd?player_id=61390  


Dataframes
season_dataframes: return the dataframes consisted of seasons and each season values
teams_dataframes: return the dataframe consisted of team names and the link to read data of a team

In [ ]:
import re
import pandas as pd
def season_dataframes(soup):
  df = pd.DataFrame()
  list = soup.find("select", id = "seasonSelect")
  for data in list.find_all("option")[1:]:
    df = df.append({"value": data["value"], "date" : data.text}, ignore_index = True)
  df.set_index("date", inplace = True)
  
  return df


In [ ]:
def teams_dataframes(soup):
  df = pd.DataFrame()
  list = soup.find("tbody")
  for data in list.find_all("td", class_ = "bull"):
    team = data.find("a")
    df = df.append({"link": team["href"], "name" : team.text}, ignore_index = True)
  df.set_index("name", inplace = True)
  return df

In [ ]:
#readCompetitionPage('https://www.soccerbase.com/tournaments/tournament.sd?comp_id=20')
getEveryCompetitionData()

                                   link               name
0   /players/player.sd?player_id=124386       Balint Demus
1    /players/player.sd?player_id=40592    John McCafferty
2    /players/player.sd?player_id=48118    Stuart McKenzie
3   /players/player.sd?player_id=127698       Tom Leighton
4    /players/player.sd?player_id=88654        Harry Milne
5    /players/player.sd?player_id=78280       Alan Redford
6    /players/player.sd?player_id=49096         Scott Ross
7    /players/player.sd?player_id=40609      Ryan Strachan
8    /players/player.sd?player_id=56997       Jordan Brown
9    /players/player.sd?player_id=52101       Fraser Fyvie
10   /players/player.sd?player_id=93060     Daniel Higgins
11   /players/player.sd?player_id=59474       Jamie Masson
12   /players/player.sd?player_id=60514       James Redman
13   /players/player.sd?player_id=72982       Conor Scully
14  /players/player.sd?player_id=121995        Connor Wood
15   /players/player.sd?player_id=72987         Blair Yu